In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

GFG Jobathon: 32

Q1: Lead Engineer

In [5]:
def leadOptions(self, N : int, E : List[int]) -> int:
    maxE: tuple[int, int] = -1, 0
    for exp in E:
        if exp == maxE[0]:
            maxE = maxE[0], maxE[1] + 1
        if exp > maxE[0]:
            maxE = exp, 0

    return maxE[1]

Codeforces: 20/C (Dijkstra)

In [ ]:
def dijkstra(src: int, adjl: dict[int, list[tuple[int, int]]]) -> dict[int, tuple[float, int]]:
    heap: list[tuple[int, int, int]] = [(1, src, 1)]
    shortest: dict[int, tuple[float, int]] = dict()
    while heap:
        curr_dist, curr, prev = heapq.heappop(heap)
        if curr not in shortest or curr_dist < shortest[curr][0]:
            shortest[curr] = (curr_dist, prev)
            for next_, next_dist in adjl.get(curr, []):
                heapq.heappush(heap, (curr_dist + next_dist, next_, curr))

    return shortest

def twentyC(V: int, E: int, edges: list[tuple[int, int, int]]) -> str:
    adjl: dict[int, list[tuple[int, int]]] = dict()
    for n1, n2, d in edges:
        n1_neighbours, n2_neighbours = adjl.get(n1, []), adjl.get(n2, [])
        n1_neighbours.append((n2, d))
        n2_neighbours.append((n1, d))
        adjl[n1], adjl[n2] = n1_neighbours, n2_neighbours

    paths = dijkstra(1, adjl)
    if V in paths:
        result: list[int] = [V]
        curr = V
        while curr != 1:
            dist, prev = paths[curr]
            result.append(prev)
            curr = prev

        result.reverse()
        return ' '.join(map(str, result))
    else:
        return '-1'

# Testing the solution
assert twentyC(5, 5, [(1, 2, 2), (2, 5, 5), (2, 3, 4), (1, 4, 1), (4, 3, 3), (3, 5, 1)]) == '1 4 3 5'